In [ ]:
%pip install transformers torch datasets trl tqdm accelerate peft

In [ ]:
# 1: 
# Импорты и зависимости

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from datasets import load_dataset
from tqdm.notebook import tqdm
import os

print("Импорты выполнены ✅")

# Проверка доступности GPU
print("Информация о системе:")
print(f"-->Доступен CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("-->Используется CPU")

In [ ]:
# 2: 
# Загрузка и подготовка данных

def load_and_format_dataset():
    """Загрузка и форматирование датасета"""
    
    print("🔄 Загружаем датасет...")
    ds = load_dataset("d0rj/alpaca-cleaned-ru", split="train")
    
    # Функция для форматирования данных в диалоговый формат
    def format_alpaca_to_chat(example):
        """Преобразует пример Alpaca в диалоговый формат для Qwen"""
        if example['input']:
            user_message = f"{example['instruction']}\n\n{example['input']}"
        else:
            user_message = example['instruction']
        
        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": example['output']}
        ]
        
        # Форматируем в строку с использованием формата Qwen
        formatted_text = ""
        for msg in messages:
            if msg["role"] == "user":
                formatted_text += f"<|im_start|>user\n{msg['content']}<|im_end|>\n"
            else:
                formatted_text += f"<|im_start|>assistant\n{msg['content']}<|im_end|>\n"
        
        return {"text": formatted_text}
    
    # Применяем форматирование к датасету
    print("🔄 Форматируем данные...")
    formatted_ds = ds.map(format_alpaca_to_chat)
    
    # Проверяем форматирование на примере
    print("\nПример форматированного текста:")
    print(formatted_ds[11]['text'][:500] + "...")
    print(f"\n✅ Размер датасета: {len(formatted_ds)} примеров")
    
    return formatted_ds

# Загружаем данные
formatted_ds = load_and_format_dataset()

In [ ]:
# 3: 
# Тестирование модели ДО обучения

def test_before_training():
    """Тестирование модели до обучения"""
    print("Тестируем модель ДО обучения...")
    
    model_id = "Qwen/Qwen2.5-0.5B"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    questions_rus = [
        "сколько планет в нашей солнечной системе?",
        "расскажи стих",
        "когда собирать крыжовник?",
        "Как быстро выучить новый язык?"
    ]
    
    for i, question in enumerate(questions_rus, 1):
        print(f"\n{i}. Вопрос: {question}")
        
        messages = [{"role": "user", "content": question}]
        
        # Используем правильный метод для применения шаблона чата
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        inputs = tokenizer(text, return_tensors="pt").to(model.device)
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
        
        response = outputs[0][inputs['input_ids'].shape[1]:]
        answer = tokenizer.decode(response, skip_special_tokens=True)
        
        print(f"Ответ: {answer}")
        print("―" * 55)

# Запустить до обучения
test_before_training()

In [ ]:
# 4: 
# Настройка SFT с LoRA

def setup_training():
    """Настройка параметров обучения с SFTConfig"""
    
    model_id = "Qwen/Qwen2.5-0.5B"
    
    # Загрузка токенизатора и модели
    print(f"🔄 Загружаем модель {model_id}...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    
    # Определяем устройство (CPU/GPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Используем устройство: {device}")
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None
    )
    
    # Конфигурация LoRA
    peft_cfg = LoraConfig(
        r=8, 
        lora_alpha=16, 
        lora_dropout=0.05,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        task_type="CAUSAL_LM",
        bias="none"
    )
    
    # Создаем SFTConfig 
    cfg = SFTConfig(
        output_dir="./qwen2.5-0.5b-sft-ru",
        dataset_text_field="text",
        max_seq_length=1024,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  
        logging_steps=50,
        num_train_epochs=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        warmup_ratio=0.03,
        report_to=None,
        run_name='qwen-lora-sft',
        save_steps=500,
        fp16=True,  # Включаем mixed precision для экономии памяти
        optim="adamw_torch",
        remove_unused_columns=False,
        packing=True,  # Экономит память
    )
    
    return model, tokenizer, peft_cfg, cfg

# Настраиваем обучение
model, tokenizer, peft_cfg, cfg = setup_training()

In [ ]:
# 5: Запуск SFT обучения с LoRA
def create_and_train_model(formatted_ds):
    """Создание и обучение модели с SFTConfig"""
    
    print("🔄 Создаем тренер...")
    
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,  # Используем tokenizer вместо processing_class
        args=cfg,
        train_dataset=formatted_ds,
        peft_config=peft_cfg,
        dataset_text_field="text",  # Добавляем явное указание поля
        max_seq_length=1024,  # Явно указываем max_seq_length
        packing=True,  # Включаем packing для экономии памяти
    )
    
    # Запускаем обучение
    print("🚀 Начинаем обучение...")
    training_result = trainer.train()
    
    # Сохраняем модель
    print("💾 Сохраняем модель...")
    trainer.save_model()
    tokenizer.save_pretrained("./qwen2.5-0.5b-sft-ru")
    
    print("✅ Обучение завершено!")
    return trainer

# Запускаем обучение
trainer = create_and_train_model(formatted_ds)

In [ ]:
# 6: Тестирование модели ПОСЛЕ обучения
def test_after_training():
    """Тестирование модели после обучения"""
    
    print("\nТестируем модель ПОСЛЕ обучения...")
    
    # Загружаем обученную модель
    model_path = "./qwen2.5-0.5b-sft-ru"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )
    
    questions_rus = [
        "сколько планет в нашей солнечной системе?",
        "расскажи стих",
        "когда собирать крыжовник?",
        "Как быстро выучить новый язык?"
    ]
    
    print("Ответы обученной модели:\n")
    
    for i, question in enumerate(tqdm(questions_rus, desc="Тестирование"), 1):
        print(f"\n{i}. Вопрос: {question}")
        
        # Форматируем сообщение
        messages = [{"role": "user", "content": question}]
        
        # Применяем шаблон чата
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
            )
        
        inputs = tokenizer(text, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        # Генерация
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
        
        # Извлекаем ответ
        response = outputs[0][inputs['input_ids'].shape[1]:]
        answer = tokenizer.decode(response, skip_special_tokens=True)
        
        print(f"Ответ: {answer.strip()}")
        print("―" * 55)

# Тестируем после обучения
test_after_training()

In [ ]:
# 7: Очистка памяти
def cleanup_memory():
    """Очистка памяти GPU"""
    import gc
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    print("✅ Память очищена")

# Запустить при необходимости
cleanup_memory()